<font color="#CA3532"><h1 align="left">**Aprendizaje por refuerzo**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Multiarmed bandits no contextuales**</h2></font>


# **Estrategia Multiarmed Bandit Gaussiano con tres acciones posibles**

- El agente puede tomar 3 acciones ("bandits"): 1, 2, 3
- El resultado de la acción i tiene un reward $r_i$
- El $r_i$ sigue una distribución gaussiana con media $m_i$ y desviación estándar $std_i$
- Los 6 parámetros ($m_1$, $std_1$, $m_2$, $std_2$, $m_3$, $std_3$) son inicialmente desconocidos para el agente
- El agente tiene que tomar decisiones una y otra vez
- Su objetivo es maximizar su beneficio

## Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Clases en python

In [ ]:
from sklearn.linear_model import LogisticRegression # LogisticRegression es una clase

In [ ]:
model1 = LogisticRegression() # creamos una instancia de LogisticRegression y la instancia se asocia a model1
model2 = LogisticRegression() # creamos OTRA instancia de LogisticRegression y la instancia se asocia a model2

In [ ]:
np.zeros # una función de la librería numpy
model1.fit # un método asociado a la instancia model1

In [ ]:
class Persona():
  def __init__(self, age, name): # método constructor. Es lo que se llama automáticamente cuando se crea una instancia
    # self: "variables internas de la instancia"
    self.edad = age # propiedad edad
    self.nombre = name #propiedad nombre
  def myprint(self): # método "myprint"
    print("Nombre:", self.nombre)
    print("Edad:  ", self.edad)
  def cumple_anyos(self):
    self.edad = self.edad + 1

In [ ]:
p1 = Persona(18, "Juan")
p2 = Persona(50, "Ana")
p1.myprint() # el () ejecuta el método. Si no, no se ejecuta

In [ ]:
p2.myprint()

In [ ]:
p1.cumple_anyos()

In [ ]:
p1.myprint()

In [ ]:
p2.myprint()

## Modelización del entorno

In [ ]:
aux = 10+5*np.random.randn(10000)
plt.hist(aux, bins=20)
print(aux.mean(), aux.std())

In [ ]:
class Entorno():
  def __init__(self, m0, std0, m1, std1, m2, std2):
    self.medias = [m0, m1, m2]
    self.stds   = [std0, std1, std2]
  def act(self, accion): # accion: a qué máquina tragaperras (0, 1, 2) se echa la moneda
    media_premios = self.medias[accion]
    std_premios   = self.stds[accion]

    return media_premios + std_premios*np.random.randn() # genero premio de una distribución de media
    # media_premios y desviación estándar std_premios

In [ ]:
# "Me vas a simular un entorno con 3 máquinas tragaperras 0, 1, 2,
# cada una de ellas con media de premio mi y desviación estándar de premio stdi"
e = Entorno(m0=1, std0=1, m1=2, std1=1, m2=3, std2=1)

In [ ]:
# premio que obtenemos en máquina 2:
e.act(2)

In [ ]:
# chequeemos si el modelo del entorno está "bien" implementado:

N = 10000 # número de tiradas que vamos a simular en cada máquina, para ver si está bien implementado



In [ ]:
def chequea_maquina(entorno, maquina, Nmonedas):
  # Simula que jugamos Nmonedas veces a la máquina i
  # Hace print de la media y std de los premios
  # saca histograma de esos premios

  #premios = []
  #for _ in range(Nmonedas):
  #  premios.append(entorno.act(maquina))
  premios = [entorno.act(maquina) for _ in range(Nmonedas)]

  print("Media:", np.mean(premios), "Std:", np.std(premios))
  plt.figure(figsize=(3,3))
  #plt.hist(premios, bins=20)
  sns.distplot(premios)

In [ ]:
chequea_maquina(entorno=e, maquina=0, Nmonedas=N)

In [ ]:
chequea_maquina(entorno=e, maquina=1, Nmonedas=N)

In [ ]:
chequea_maquina(entorno=e, maquina=2, Nmonedas=N)

In [ ]:
def chequea_maquinas(entorno, Nmonedas):
  plt.figure(figsize=(3,3))
  for maquina in range(3):
    premios = [entorno.act(maquina) for _ in range(Nmonedas)]
    sns.distplot(premios)

In [ ]:
chequea_maquinas(entorno=e, Nmonedas=10000)

# 0- Agente que separa completamente exploración y explotación

In [ ]:
for _ in range(100):
  print(np.random.choice(3))

In [ ]:
np.argmax([10,20,3])

In [ ]:
# Agente0: las primeras Nexploracion monedas las echa a máquinas al azar
# las restantes monedas las echa a la máquina con mejor valoración (mejor Q, premio promedio observado)
class Agente0():
  def __init__(self, Nexploracion): # número de monedas que dedicamos a explorar.
    # Posteriormente, dedicaremos todas las monedas a explotar (máquina que haya resultado mejor en exploración)
    self.n_exploracion = Nexploracion
    self.ntotal = 0 # número de monedas jugadas hasta ahora, en total
    self.nveces = [0, 0, 0] # lo mismo, pero en cada máquina en particular
    self.suma_premios = [0, 0, 0] # premios acumulados en cada una de las máquinas
    self.Qs = [-1000, -1000, -1000]    # premios promedios en cada máquina

  def choose_action(self):
    # las primeras Nexploracion monedas las echa a máquinas al azar
    # las restantes monedas las echa a la máquina con mejor valoración (mejor Q, premio promedio observado)
    if self.ntotal < self.n_exploracion:
      accion_elegida = np.random.choice(3) # elijo 0, 1 o 2 aleatoriamente
    else: #elijo la máquina con mejor Q
      accion_elegida = np.argmax(self.Qs)

    return accion_elegida

  def update(self, accion, premio): # (premio = reward)
    self.ntotal += 1 # equivalente a self.ntotal = self.ntotal + 1
    self.nveces[accion] += 1
    self.suma_premios[accion] += premio # actualizamos premio acumulado en máquina "accion"
    self.Qs[accion] = self.suma_premios[accion]/self.nveces[accion]

In [ ]:
agente = Agente0(Nexploracion=100)

In [ ]:
agente.ntotal

In [ ]:
agente.choose_action() # máquina elegida por agente

In [ ]:
e.act(2)

In [ ]:
agente.nveces

In [ ]:
agente.suma_premios

In [ ]:
agente.Qs

In [ ]:
agente.update(2, 2.824540848890503)

In [ ]:
def experimento_simplificado(agente, entorno, N): # sin extraer estadísticas
  for i in range(N): # iteración sobre N casos
    accion = agente.choose_action() # ¿Qué acción debería elegir?
    reward = entorno.act(accion)    # Realizamos esa acción y observamos el reward
    agente.update(accion, reward)   # El agente aprende con esa acción y ese reward

In [ ]:
def experimento(agente, entorno, N): # N: total de monedas que va a echar el agente, incluyendo exploración y explotación
  reward_total_actual       = 0
  reward_total_historico    = []
  reward_promedio_historico = []
  regret_historico          = []
  Qs_0 = []
  Qs_1 = []
  Qs_2 = []
  acciones = []

  for i in range(N):
    accion = agente.choose_action()
    reward = entorno.act(accion)
    agente.update(accion, reward)

    reward_total_actual += reward
    reward_total_historico.append(reward)
    reward_promedio_historico.append(reward_total_actual/(i+1))
    # 3*(i+1) es el premio total estimado que tendría si hubiera hecho trampas
    # (me dicen que la mejor máquina es la tercera), ya que da un premio en promedio
    # de 3
    # i+1: número de monedas que llevo echadas
    regret_historico.append(3*(i+1) - reward_total_actual)

    Qs_0.append(agente.Qs[0])
    Qs_1.append(agente.Qs[1])
    Qs_2.append(agente.Qs[2])
    acciones.append(accion)

  # sacamos algunas gráficas:
  plt.plot(reward_total_historico)
  plt.title("reward total histórico")
  plt.show() # no me mezcla esta gráfica con la siguiente

  plt.plot(reward_promedio_historico)
  plt.title("reward promedio histórico")
  plt.show()

  plt.plot(regret_historico)
  plt.title("regret total histórico")
  plt.show()

  print("Qs finales:", agente.Qs)
  print("Premio promedio por moneda (incluyendo exploración y explotación):",
        reward_promedio_historico[-1])
  return Qs_0, Qs_1, Qs_2, acciones

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

entorno = Entorno(m0=1, std0=1, m1=2, std1=1, m2=3, std2=1)
agente  = Agente0(Nexploracion=500)
experimento(agente, entorno, N=10000)

# 1- Agente Greedy

### Implementación del agente

In [ ]:
class AgenteGreedy():
  def __init__(self):
    # Posteriormente, dedicaremos todas las monedas a explotar (máquina que haya resultado mejor en exploración)
    self.nveces = [0, 0, 0] # lo mismo, pero en cada máquina en particular
    self.suma_premios = [0, 0, 0] # premios acumulados en cada una de las máquinas
    self.Qs = [-1000, -1000, -1000]    # premios promedios en cada máquina

  def choose_action(self):
    #elijo la máquina con mejor Q
    accion_elegida = np.argmax(self.Qs)

    return accion_elegida

  def update(self, accion, premio): # (premio = reward)
    self.nveces[accion] += 1
    self.suma_premios[accion] += premio # actualizamos premio acumulado en máquina "accion"
    self.Qs[accion] = self.suma_premios[accion]/self.nveces[accion]

### Simulación de interacción del algoritmo Greedy con el entorno

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

entorno = Entorno(m0=1, std0=1, m1=2, std1=1, m2=3, std2=1)
agente  = AgenteGreedy()
experimento(agente, entorno, N=10000);

# 4- Agente OIV (Optimistic Initial Values)

In [ ]:
class AgenteOIV():
  def __init__(self, premio_inicial=10): # premio_inicial: nos inventamos que cada una de las máquinas nos ha dado
    # ese premio con 1 moneda inventada
    # Posteriormente, dedicaremos todas las monedas a explotar (máquina que haya resultado mejor en exploración)
    self.nveces = [1, 1, 1] # lo mismo, pero en cada máquina en particular
    self.suma_premios = [premio_inicial, premio_inicial, premio_inicial] # premios acumulados en cada una de las máquinas
    self.Qs = [premio_inicial, premio_inicial, premio_inicial]    # premios promedios en cada máquina

  def choose_action(self):
    #elijo la máquina con mejor Q
    accion_elegida = np.argmax(self.Qs)

    return accion_elegida

  def update(self, accion, premio): # (premio = reward)
    self.nveces[accion] += 1
    self.suma_premios[accion] += premio # actualizamos premio acumulado en máquina "accion"
    self.Qs[accion] = self.suma_premios[accion]/self.nveces[accion]

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

entorno = Entorno(m0=1, std0=1, m1=2, std1=1, m2=3, std2=1)
agente  = AgenteOIV()
Qs_0, Qs_1, Qs_2, acciones = experimento(agente, entorno, N=1000)

In [ ]:
plt.plot(Qs_0, label="Q máquina 0")
plt.plot(Qs_1, label="Q máquina 1")
plt.plot(Qs_2, label="Q máquina 2")
plt.legend();

In [ ]:
plt.plot(acciones, ".");

# 2- Agente $\epsilon$-Greedy

In [ ]:
# si yo quiero "lanzar una moneda" y que el epsilon de las veces salga cara:
epsilon = 0.2
np.random.rand() < epsilon # rand(): número al azar entre 0 y 1

In [ ]:
np.random.rand()

In [ ]:
np.random.rand(100)

In [ ]:
(np.random.rand(10000) < 0.2).sum()/10000

### Implementación del agente

In [ ]:
class AgenteEpsilonGreedy():
  def __init__(self, epsilon): # 0: nunca se explora. 0.1: se explora el 10% de las veces. 0.9: se explora el 90% de las veces
    # Posteriormente, dedicaremos todas las monedas a explotar (máquina que haya resultado mejor en exploración)
    self.epsilon = epsilon
    self.nveces = [0, 0, 0] # lo mismo, pero en cada máquina en particular
    self.suma_premios = [0, 0, 0] # premios acumulados en cada una de las máquinas
    self.Qs = [-10, -10, -10]    # premios promedios en cada máquina

  def choose_action(self):
    if np.random.rand() < self.epsilon: # exploro al azar:
      accion_elegida = np.random.choice(3) # elijo 0, 1 o 2 aleatoriamente
    else: #elijo la máquina con mejor Q
      accion_elegida = np.argmax(self.Qs)

    return accion_elegida

  def update(self, accion, premio): # (premio = reward)
    self.nveces[accion] += 1
    self.suma_premios[accion] += premio # actualizamos premio acumulado en máquina "accion"
    self.Qs[accion] = self.suma_premios[accion]/self.nveces[accion]

### Simulación de interacción del algoritmo $\epsilon$-Greedy con el entorno

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

entorno = Entorno(m0=1, std0=1, m1=2, std1=1, m2=3, std2=1)
agente  = AgenteEpsilonGreedy(epsilon=0.2)
Qs_0, Qs_1, Qs_2, acciones = experimento(agente, entorno, N=1000)

In [ ]:
plt.plot(Qs_0, label="Q máquina 0")
plt.plot(Qs_1, label="Q máquina 1")
plt.plot(Qs_2, label="Q máquina 2")
plt.legend();

In [ ]:
plt.plot(acciones, ".");

# 3- Agente Decaying $\epsilon$-Greedy

In [ ]:
def epsilon_dinamico(t, k):
    return min(1, k/t)

tiempos = range(1,201)
k = 50
epsilons = [epsilon_dinamico(t,k) for t in tiempos]
plt.plot(tiempos, epsilons);

### Implementación del agente

Una estrategia para enfriar el epsilon (factor de exploración):

In [ ]:
class AgenteDecayingEpsilonGreedy():
  def __init__(self, k): # 0: nunca se explora. 0.1: se explora el 10% de las veces. 0.9: se explora el 90% de las veces
    # Posteriormente, dedicaremos todas las monedas a explotar (máquina que haya resultado mejor en exploración)
    self.k = k
    self.ntotal = 0 # número de monedas jugadas hasta ahora, en total
    self.nveces = [0, 0, 0] # lo mismo, pero en cada máquina en particular
    self.suma_premios = [0, 0, 0] # premios acumulados en cada una de las máquinas
    self.Qs = [-10, -10, -10]    # premios promedios en cada máquina

  def choose_action(self):
    epsilon = epsilon_dinamico(self.ntotal+1, self.k)
    if np.random.rand() < epsilon: # exploro al azar:
      accion_elegida = np.random.choice(3) # elijo 0, 1 o 2 aleatoriamente
    else: #elijo la máquina con mejor Q
      accion_elegida = np.argmax(self.Qs)

    return accion_elegida

  def update(self, accion, premio): # (premio = reward)
    self.ntotal += 1
    self.nveces[accion] += 1
    self.suma_premios[accion] += premio # actualizamos premio acumulado en máquina "accion"
    self.Qs[accion] = self.suma_premios[accion]/self.nveces[accion]

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

entorno = Entorno(m0=1, std0=1, m1=2, std1=1, m2=3, std2=1)
agente  = AgenteDecayingEpsilonGreedy(k=10)
Qs_0, Qs_1, Qs_2, acciones = experimento(agente, entorno, N=1000)

In [ ]:
plt.plot(Qs_0, label="Q máquina 0")
plt.plot(Qs_1, label="Q máquina 1")
plt.plot(Qs_2, label="Q máquina 2")
plt.legend();

In [ ]:
plt.plot(acciones, ".");

### Simulación de interacción del algoritmo Decaying $\epsilon$-Greedy con el entorno

Otra forma:

In [ ]:
def epsilon_dinamico2(t, k):
    return np.exp(-k*(t-1))

k = 0.1
tiempos = range(1,201)
epsilons = [epsilon_dinamico2(t,k) for t in tiempos]
plt.plot(tiempos, epsilons);

In [ ]:
class AgenteDecayingEpsilonGreedy2():
  def __init__(self, k): # 0: nunca se explora. 0.1: se explora el 10% de las veces. 0.9: se explora el 90% de las veces
    # Posteriormente, dedicaremos todas las monedas a explotar (máquina que haya resultado mejor en exploración)
    self.k = k
    self.ntotal = 0 # número de monedas jugadas hasta ahora, en total
    self.nveces = [0, 0, 0] # lo mismo, pero en cada máquina en particular
    self.suma_premios = [0, 0, 0] # premios acumulados en cada una de las máquinas
    self.Qs = [-10, -10, -10]    # premios promedios en cada máquina

  def choose_action(self):
    epsilon = epsilon_dinamico2(self.ntotal+1, self.k)
    if np.random.rand() < epsilon: # exploro al azar:
      accion_elegida = np.random.choice(3) # elijo 0, 1 o 2 aleatoriamente
    else: #elijo la máquina con mejor Q
      accion_elegida = np.argmax(self.Qs)

    return accion_elegida

  def update(self, accion, premio): # (premio = reward)
    self.ntotal += 1
    self.nveces[accion] += 1
    self.suma_premios[accion] += premio # actualizamos premio acumulado en máquina "accion"
    self.Qs[accion] = self.suma_premios[accion]/self.nveces[accion]

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

entorno = Entorno(m0=1, std0=1, m1=2, std1=1, m2=3, std2=1)
agente  = AgenteDecayingEpsilonGreedy2(k=0.1)
Qs_0, Qs_1, Qs_2, acciones = experimento(agente, entorno, N=1000)

In [ ]:
plt.plot(Qs_0, label="Q máquina 0")
plt.plot(Qs_1, label="Q máquina 1")
plt.plot(Qs_2, label="Q máquina 2")
plt.legend();

In [ ]:
plt.plot(acciones, ".");

# 5- Agente UCB1

In [ ]:
class AgenteUCB1():
  def __init__(self):
    # Posteriormente, dedicaremos todas las monedas a explotar (máquina que haya resultado mejor en exploración)
    self.nveces = [0, 0, 0] # lo mismo, pero en cada máquina en particular
    self.suma_premios = [0, 0, 0] # premios acumulados en cada una de las máquinas
    self.Qs = [0, 0, 0]    # premios promedios en cada máquina
    self.t = 1

  def choose_action(self):
    #elijo la máquina con mejor UCB = Q + U; U=sqrt(2*log(t)/nveces)
    ucbs = [0,0,0]
    for i in range(3):
      if self.nveces[i]==0:
        ucbs[i] = 10000
      else:
        ucbs[i] = self.Qs[i] + np.sqrt(2*np.log(self.t)/self.nveces[i])

    accion_elegida = np.argmax(ucbs)

    return accion_elegida

  def update(self, accion, premio): # (premio = reward)
    self.nveces[accion] += 1
    self.suma_premios[accion] += premio # actualizamos premio acumulado en máquina "accion"
    self.Qs[accion] = self.suma_premios[accion]/self.nveces[accion]
    self.t += 1

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

entorno = Entorno(m0=1, std0=1, m1=1.9, std1=1, m2=2, std2=1)

chequea_maquinas(entorno=entorno, Nmonedas=10000)
plt.show()

agente  = AgenteUCB1()
Qs_0, Qs_1, Qs_2, acciones = experimento(agente, entorno, N=1000)

In [ ]:
plt.plot(Qs_0, label="Q máquina 0")
plt.plot(Qs_1, label="Q máquina 1")
plt.plot(Qs_2, label="Q máquina 2")
plt.legend();

In [ ]:
plt.plot(acciones, ".");